In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import polars as pl
import torch
from pypalettes import get_hex
#pip install git+https://github.com/KindXiaoming/pykan.git
import sys
sys.path.append(r"C:\Users\TESTER\OneDrive\Documentos\GitHub\Projects_Zone")

import utils as u

ModuleNotFoundError: No module named 'polars'

In [ ]:
test_data = pd.read_csv(r""C:\Users\ADMIN\Desktop\Thesis Space Desktop\UNSW_NB15_testing-set.csv"")
train_data = pd.read_csv(r"C:\Users\ADMIN\Desktop\Thesis Space Desktop\UNSW_NB15_training-set.csv")

In [ ]:
len(train_data["proto"].unique()), len(test_data["proto"].unique())

In [ ]:
columns_numeric = train_data.select_dtypes(include="number").drop(columns="id").columns

In [ ]:
len(columns_numeric)

In [ ]:
columns_numeric = [
    col for col in columns_numeric
    if not len(train_data[col].value_counts()) < 3
]

In [ ]:
len(columns_numeric)

In [ ]:
train_data.set_index("id", inplace=True)

In [ ]:
test_data.set_index("id", inplace=True)

In [ ]:
pd.set_option('display.max_columns', 120)
train_data.head(2)

# Data Preprocessing

## Outlier Treatment

In [ ]:
fig, axes = plt.subplots(20,2, figsize=(20,40), tight_layout=True)
axes = axes.flatten()

for col,ax in zip(columns_numeric, axes):
    g = sns.boxplot(data = train_data, y = col, x = train_data["attack_cat"],hue = train_data["attack_cat"], ax = ax)
    g.set_title(f"Boxplot for {col}")
[fig.delaxes(ax) for ax in axes.flatten() if not ax.has_data()]

plt.show()
    

In [ ]:
#class specific

def treat_outliers(dataframe, columns_numeric, isTest=False):
    """
    Treats outliers in the dataset for the specified numeric columns.

    On the training set:
        - Identifies outliers using the IQR method for each column grouped by "attack_cat".
        - Replaces outliers with the mean value of the column grouped by "attack_cat".
        - Stores the IQR bounds for each column and class.

    On the test set:
        - Uses the stored IQR bounds from training for each column and class.
        - Replaces outliers with the mean value of the column grouped by "attack_cat".

    Parameters:
        dataframe (pd.DataFrame): The dataset to process.
        columns_numeric (list): List of numeric columns to focus on for outlier treatment.
        isTest (bool): Whether the function is applied to the test set.

    Returns:
        pd.DataFrame: The processed dataset with outliers treated.
    """
    if not hasattr(treat_outliers, "iqr_bounds"):
        treat_outliers.iqr_bounds = {}  # Store IQR bounds for training (keyed by column and class)

    if isTest and not treat_outliers.iqr_bounds:
        raise ValueError("IQR bounds are not set. Train the function on the training data first.")

    print("Treating outliers...")
    
    # Ensure all numeric columns are cast to float to handle outliers consistently
    dataframe[columns_numeric] = dataframe[columns_numeric].astype(float)

    # Process each numeric column
    for col in columns_numeric:
        if not isTest:
            # Initialize dictionary for column if not already present
            treat_outliers.iqr_bounds[col] = {}

            # Group by "attack_cat" to calculate class-specific IQR bounds
            for attack_cat, group in dataframe.groupby("attack_cat"):
                q1 = group[col].quantile(0.25)
                q3 = group[col].quantile(0.75)
                iqr = q3 - q1
                lower_bound = q1 - 1.5 * iqr
                upper_bound = q3 + 1.5 * iqr
                treat_outliers.iqr_bounds[col][attack_cat] = (lower_bound, upper_bound)

                # Identify outliers for this class
                outliers = (group[col] < lower_bound) | (group[col] > upper_bound)

                # Replace outliers with the class mean
                replacement_mean = group[col].mean()
                dataframe.loc[group[outliers].index, col] = replacement_mean
        else:
            # Use stored IQR bounds for test set
            for attack_cat, bounds in treat_outliers.iqr_bounds[col].items():
                lower_bound, upper_bound = bounds
        
                # Get test data rows for this attack_cat
                class_rows = dataframe[dataframe["attack_cat"] == attack_cat]
        
                # Identify outliers
                outliers = (class_rows[col] < lower_bound) | (class_rows[col] > upper_bound)
        
                # Replace outliers with the training class mean
                replacement_mean = dataframe[dataframe["attack_cat"] == attack_cat][col].mean()
                dataframe.loc[class_rows[outliers].index, col] = replacement_mean


    print("Outlier treatment completed.")
    return dataframe


In [ ]:
train_data = treat_outliers(train_data, columns_numeric)

In [ ]:
test_data = treat_outliers(test_data, columns_numeric,isTest=True)

In [ ]:
fig, axes = plt.subplots(20,2, figsize=(20,40), tight_layout=True)
axes = axes.flatten()

for col,ax in zip(columns_numeric, axes):
    g = sns.boxplot(data = train_data, y = col, x = train_data["attack_cat"],hue = train_data["attack_cat"], ax = ax)
    g.set_title(f"Boxplot for {col}")
[fig.delaxes(ax) for ax in axes.flatten() if not ax.has_data()]

plt.show()

A lot more quantiles are displayed now !!

In [ ]:
train_cat = train_data.select_dtypes(exclude=[np.number])
train_cat.describe(include='all')

In [ ]:
# Function to reduce categories in categorical columns while preserving the target column
def reduce_categories(train_data, test_data, target_column, threshold=6, debug=False):
    # Select categorical columns in train and test data, excluding the target column
    train_cat = train_data.select_dtypes(exclude=[np.number]).drop(columns=[target_column], errors='ignore')
    test_cat = test_data.select_dtypes(exclude=[np.number]).drop(columns=[target_column], errors='ignore')
    
    # Iterate through categorical columns
    for feature in train_cat.columns:
        if debug:
            print(f"Processing feature: {feature}")
            print(f"Number of unique categories before reduction: {train_cat[feature].nunique()}")
            print('----------------------------------------------------')
        
        # Check if the number of unique categories exceeds the threshold
        if train_cat[feature].nunique() > threshold:
            # Identify the top categories in the train set
            top_categories = train_data[feature].value_counts().head(threshold).index
            
            # Reduce train and test data to these top categories, replacing others with '-'
            train_data[feature] = np.where(train_data[feature].isin(top_categories), train_data[feature], '-')
            test_data[feature] = np.where(test_data[feature].isin(top_categories), test_data[feature], '-')
    
    # Return the modified train and test datasets
    return train_data, test_data

# Apply the function to your train and test datasets
train_data, test_data = reduce_categories(train_data, test_data, target_column='attack_cat', threshold=6, debug=False)

# Check the resulting categorical columns
train_cat = train_data.select_dtypes(exclude=[np.number])
test_cat = test_data.select_dtypes(exclude=[np.number])

print("Train categorical summary after reduction:")
print(train_cat.describe(include='all'))

print("\nTest categorical summary after reduction:")
print(test_cat.describe(include='all'))

## Normalizing and Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [ ]:
numeric = train_data.select_dtypes(include="number").columns

normalizer = MinMaxScaler().set_output(transform="pandas")
scaler = StandardScaler().set_output(transform="pandas")
r_scaler = RobustScaler().set_output(transform="pandas")

train_data[numeric] = r_scaler.fit_transform(train_data[numeric])
test_data[numeric] = r_scaler.transform(test_data[numeric])

train_data[numeric] = scaler.fit_transform(train_data[numeric])
test_data[numeric] = scaler.transform(test_data[numeric])

train_data[numeric] = normalizer.fit_transform(train_data[numeric])
test_data[numeric] = normalizer.transform(test_data[numeric])

In [ ]:
train_data.shape, test_data.shape

In [ ]:
len(train_data["state"].unique()), len(test_data["state"].unique())

## Treating Skewness

In [ ]:
import pandas as pd
import numpy as np

# Set skewness threshold (e.g., |skewness| > 1 is considered highly skewed)
skew_threshold = 1

numeric_cols = train_data.select_dtypes(include = "number").columns

# Select numeric columns
# Calculate skewness for numeric columns
skewness = train_data[numeric_cols].skew()

# Identify columns with skewness above the threshold
skewed_cols = skewness[skewness.abs() > skew_threshold].index
print("Skewed columns:", skewed_cols)

# Apply log transformation to skewed columns
for col in skewed_cols:
    # Add 1 to avoid issues with log(0)
    train_data[col] = np.log1p(train_data[col])
    test_data[col] = np.log1p(test_data[col])  # Apply same transformation to test data


# Feature Engineering

## OneHotEncoding

In [ ]:
# Separate features and labels
train_data_X = train_data.drop(columns=["attack_cat", "label"])
train_data_y = train_data[["attack_cat", "label"]]

test_data_X = test_data.drop(columns=["attack_cat", "label"])
test_data_y = test_data[["attack_cat", "label"]]

# Identify categorical columns
categorical_cols = train_data_X.select_dtypes(include="object").columns
print(categorical_cols)  # Verify the categorical columns

# Use OneHotEncoder with handle_unknown='ignore' to ensure consistency
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define and fit the encoder on training data
encoder = OneHotEncoder(handle_unknown='ignore', drop='first')  # Drop first category to avoid dummy variable trap
ct = ColumnTransformer(transformers=[('encoder', encoder, categorical_cols)], remainder='passthrough')

# Fit the encoder on train data
train_data_X_encoded = ct.fit_transform(train_data_X)

# Transform the test data using the same encoder
test_data_X_encoded = ct.transform(test_data_X)

# Convert the encoded data back to DataFrame for alignment
import pandas as pd
train_data_X_encoded = pd.DataFrame(train_data_X_encoded, columns=ct.get_feature_names_out(), index=train_data_X.index)
test_data_X_encoded = pd.DataFrame(test_data_X_encoded, columns=ct.get_feature_names_out(), index=test_data_X.index)

# Ensure consistent columns between train and test
train_data_X, test_data_X = train_data_X_encoded.align(test_data_X_encoded, join="outer", axis=1, fill_value=0)

In [ ]:
for index, column_name in enumerate(test_data_X.columns):
    print(index, column_name)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Ensure you work with copies of the DataFrames
train_data_X = train_data_X.copy()
test_data_X = test_data_X.copy()

# Filter only numeric columns for scaling
numeric_cols_train = train_data_X.select_dtypes(include=['number']).columns
numeric_cols_test = test_data_X.select_dtypes(include=['number']).columns

# Ensure numeric columns are cast to float before scaling
train_data_X[numeric_cols_train] = train_data_X[numeric_cols_train].astype(float)
test_data_X[numeric_cols_test] = test_data_X[numeric_cols_test].astype(float)

# Initialize scalers
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

# Apply StandardScaler to train_data
train_data_scaled = standard_scaler.fit_transform(train_data_X[numeric_cols_train])
train_data_X.loc[:, numeric_cols_train] = train_data_scaled

# Apply StandardScaler to test_data
test_data_scaled = standard_scaler.transform(test_data_X[numeric_cols_test])
test_data_X.loc[:, numeric_cols_test] = test_data_scaled

# Apply MinMaxScaler to train_data
train_data_scaled = minmax_scaler.fit_transform(train_data_X[numeric_cols_train])
train_data_X.loc[:, numeric_cols_train] = train_data_scaled

# Apply MinMaxScaler to test_data
test_data_scaled = minmax_scaler.transform(test_data_X[numeric_cols_test])
test_data_X.loc[:, numeric_cols_test] = test_data_scaled


### Converting data to required format

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_data_X = pd.DataFrame(scaler.fit_transform(train_data_X), columns=train_data_X.columns)
test_data_X = pd.DataFrame(scaler.transform(test_data_X), columns=test_data_X.columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

train_labels_encoded = pd.DataFrame(label_encoder.fit_transform(train_data_y["attack_cat"]))
test_labels_encoded = pd.DataFrame(label_encoder.transform(test_data_y["attack_cat"]))

In [ ]:
train_data_X.to_csv(r"C:\Users\TESTER\OneDrive\Ambiente de Trabalho\Thesis Folder\Data After Preprocess\processed_train_data_X_data.csv", index=False)
test_data_X.to_csv(r"C:\Users\TESTER\OneDrive\Ambiente de Trabalho\Thesis Folder\Data After Preprocess\processed_test_data_X_data.csv", index=False)
train_labels_encoded.to_csv(r"C:\Users\TESTER\OneDrive\Ambiente de Trabalho\Thesis Folder\Data After Preprocess\processed_train_data_y_data.csv", index=False)
test_labels_encoded.to_csv(r"C:\Users\TESTER\OneDrive\Ambiente de Trabalho\Thesis Folder\Data After Preprocess\processed_test_data_y_data.csv", index=False)

In [ ]:
original_class_labels = label_encoder.classes_

import pickle
from sklearn.preprocessing import LabelEncoder

# Assuming you have a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the LabelEncoder (if not already fitted)
label_encoder.fit(['class1', 'class2', 'class3'])

# Save the LabelEncoder object to a file
with open(r"C:\Users\TESTER\OneDrive\Ambiente de Trabalho\Thesis Folder\Data After Preprocess\label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# Save the original class labels to a CSV file (optional)
original_class_labels = label_encoder.classes_
original_class_labels.to_csv(r"C:\Users\TESTER\OneDrive\Ambiente de Trabalho\Thesis Folder\Data After Preprocess\original_class_labels.csv", index=False)